## Mnist

In [1]:
import numpy as np
import torch
import matplotlib.pyplot as plt
from torch import nn, optim
from torch.autograd import Variable
from torchvision import datasets, transforms
from torch.utils.data import DataLoader

In [2]:
train_dataset = datasets.MNIST(root='/Users/qi/Documents/cood/ML/dataBase/', 
                               train=True,
                               transform = transforms.ToTensor(),
                               download=True)

test_dataset = datasets.MNIST(root='/Users/qi/Documents/cood/ML/dataBase/', 
                               train=False,
                               transform = transforms.ToTensor(),
                               download=True)

In [3]:
batch_size = 64

In [4]:
train_loader = DataLoader(dataset=train_dataset, 
                          batch_size = batch_size,
                          shuffle=True)

test_loader = DataLoader(dataset=test_dataset, 
                         batch_size = batch_size,
                         shuffle=True)

### 定义网络结构

In [5]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(784, 10)
        # 64，10 dim=1 代表着用第一个维度进行 softmax 转换，即 10
        self.softmax = nn.Softmax(dim=1)
        
        
    def forward(self, x):
        x = x.view(x.size()[0], -1)
        x = self.fc1(x)
        x = self.softmax(x)
        return x

In [6]:
LR = 0.5
model = Net()
mse_loss = nn.MSELoss()
optimizer = optim.SGD(model.parameters(), LR)

In [7]:
def train():
    for i, data in enumerate(train_loader):
        inputs, labels = data
        # 独热编码
        labels = labels.reshape(-1,1)
        # tensor.scatter(dim, index, src)
        # dim 对哪个维度进行独热编码
        # index 要将 src 中的值放到 tensor 的哪个位置
        # src 插入 index 的数值
        one_hot = torch.zeros(inputs.shape[0],10).scatter(1, labels, 1)
        out = model(inputs)
        # 这两个数值的维度要保持一致
        loss = mse_loss(out, one_hot)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

In [8]:
def test():
    corrent = 0
    for i, data in enumerate(test_loader):
        inputs, labels = data
        out = model(inputs)
        # 返回最大值，及最大值的索引 1代表需要计算的维度
        _, predict = torch.max(out, 1)
        corrent += (predict == labels).sum(0)
    print('Test acc {0}'.format(corrent.item()/len(test_dataset)))

In [9]:
for epoch in range(10):
    print('epoch ',epoch)
    train()
    test()

epoch  0
Test acc 0.8886
epoch  1
Test acc 0.9014
epoch  2
Test acc 0.9085
epoch  3
Test acc 0.9122
epoch  4
Test acc 0.9141
epoch  5
Test acc 0.9154
epoch  6
Test acc 0.9171
epoch  7
Test acc 0.9188
epoch  8
Test acc 0.9191
epoch  9
Test acc 0.9207
